# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 15 2022 11:56AM,246526,10,Enova-10643,Emerginnova,Released
1,Sep 15 2022 11:56AM,246526,10,Enova-10644,Emerginnova,Released
2,Sep 15 2022 11:56AM,246526,10,Enova-10645,Emerginnova,Released
3,Sep 15 2022 11:55AM,246525,10,AERO0915-01,Aerogen LTD.,Released
4,Sep 15 2022 11:52AM,246523,10,8529987.1,Yusen – 3D Matrix,Released
5,Sep 15 2022 11:43AM,246522,10,0085983039,ISDIN Corporation,Released
6,Sep 15 2022 11:43AM,246522,10,0085983040,ISDIN Corporation,Released
7,Sep 15 2022 11:43AM,246522,10,0085983042,ISDIN Corporation,Released
8,Sep 15 2022 11:43AM,246522,10,0085983041,ISDIN Corporation,Released
9,Sep 15 2022 11:43AM,246522,10,0085983070,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
42,246521,Released,24
43,246522,Released,18
44,246523,Released,1
45,246525,Released,1
46,246526,Released,3


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246521,NaN,NaN,24.0
246522,NaN,NaN,18.0
246523,NaN,NaN,1.0
246525,NaN,NaN,1.0
246526,NaN,NaN,3.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246067,0.0,1.0,0.0
246068,0.0,0.0,1.0
246319,0.0,0.0,1.0
246324,0.0,0.0,1.0
246325,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246067,0,1,0
246068,0,0,1
246319,0,0,1
246324,0,0,1
246325,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246067,0,1,0
1,246068,0,0,1
2,246319,0,0,1
3,246324,0,0,1
4,246325,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246067,,1,
1,246068,,,1
2,246319,,,1
3,246324,,,1
4,246325,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 15 2022 11:56AM,246526,10,Emerginnova
3,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.
4,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix
5,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation
23,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc."
47,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc."
48,Sep 15 2022 11:24AM,246519,17,Beach Products Inc
49,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions
50,Sep 15 2022 11:18AM,246516,16,Sartorius Stedim North America
51,Sep 15 2022 11:11AM,246515,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 15 2022 11:56AM,246526,10,Emerginnova,,,3
1,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,,,1
2,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,,,1
3,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,,,18
4,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",,,24
5,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc.",,,1
6,Sep 15 2022 11:24AM,246519,17,Beach Products Inc,,,1
7,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions,,,1
8,Sep 15 2022 11:18AM,246516,16,Sartorius Stedim North America,,1,
9,Sep 15 2022 11:11AM,246515,15,"Virtus Pharmaceuticals, LLC",,,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:56AM,246526,10,Emerginnova,3,,
1,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,1,,
2,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1,,
3,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18,,
4,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24,,
5,Sep 15 2022 11:24AM,246518,10,"Citieffe, Inc.",1,,
6,Sep 15 2022 11:24AM,246519,17,Beach Products Inc,1,,
7,Sep 15 2022 11:20AM,246517,16,Sartorius Bioprocess Solutions,1,,
8,Sep 15 2022 11:18AM,246516,16,Sartorius Stedim North America,,1,
9,Sep 15 2022 11:11AM,246515,15,"Virtus Pharmaceuticals, LLC",5,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:56AM,246526,10,Emerginnova,3,,
1,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,1,,
2,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1,,
3,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18,,
4,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:56AM,246526,10,Emerginnova,3.0,NaN,NaN
1,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,1.0,NaN,NaN
2,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1.0,NaN,NaN
3,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18.0,NaN,NaN
4,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 15 2022 11:56AM,246526,10,Emerginnova,3.0,0.0,0.0
1,Sep 15 2022 11:55AM,246525,10,Aerogen LTD.,1.0,0.0,0.0
2,Sep 15 2022 11:52AM,246523,10,Yusen – 3D Matrix,1.0,0.0,0.0
3,Sep 15 2022 11:43AM,246522,10,ISDIN Corporation,18.0,0.0,0.0
4,Sep 15 2022 11:35AM,246521,10,"Methapharm, Inc.",24.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5176434,112.0,20.0,7.0
15,739542,20.0,32.0,0.0
16,1478950,5.0,1.0,0.0
17,246519,1.0,0.0,0.0
19,985979,2.0,2.0,0.0
20,246497,0.0,3.0,2.0
21,1477436,5.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5176434,112.0,20.0,7.0
1,15,739542,20.0,32.0,0.0
2,16,1478950,5.0,1.0,0.0
3,17,246519,1.0,0.0,0.0
4,19,985979,2.0,2.0,0.0
5,20,246497,0.0,3.0,2.0
6,21,1477436,5.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,112.0,20.0,7.0
1,15,20.0,32.0,0.0
2,16,5.0,1.0,0.0
3,17,1.0,0.0,0.0
4,19,2.0,2.0,0.0
5,20,0.0,3.0,2.0
6,21,5.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,112.0
1,15,Released,20.0
2,16,Released,5.0
3,17,Released,1.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,17,19,20,21
Status,,,,,,,
Completed,7.0,0.0,0.0,0.0,0.0,2.0,0.0
Executing,20.0,32.0,1.0,0.0,2.0,3.0,1.0
Released,112.0,20.0,5.0,1.0,2.0,0.0,5.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,17,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,20.0,32.0,1.0,0.0,2.0,3.0,1.0
2,Released,112.0,20.0,5.0,1.0,2.0,0.0,5.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,17,19,20,21
0,Completed,7.0,0.0,0.0,0.0,0.0,2.0,0.0
1,Executing,20.0,32.0,1.0,0.0,2.0,3.0,1.0
2,Released,112.0,20.0,5.0,1.0,2.0,0.0,5.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()